In [1]:
import numpy as np
import pandas as pd
import random
from openpyxl import load_workbook

In [2]:
wb = load_workbook(filename= 'Dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','S','A','C','UUCW','TCF','ECF','Real','Real_Effort']
col1 = pd.DataFrame(col)

col1.insert(9,"Desired_UUCW", 0 , True)
col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,0
2,8,4,20,15,445,0.99,0.99,398.1,7962,0
3,9,1,5,20,355,1.03,0.8,396.75,7935,0
4,8,5,10,15,350,0.9,0.91,390.25,7805,0
5,8,1,10,16,345,0.9,0.91,387.9,7758,0
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,0
68,17,1,23,22,565,1.03,0.8,289.55,5791,0
69,19,5,30,10,475,0.95,0.92,289.1,5782,0
70,6,5,15,5,250,1,0.92,288.9,5778,0


In [3]:
for index, row in col1.iterrows():
    UAW = col1.loc[index,"UAW"]
    UUCW = col1.loc[index,"UUCW"]
    TCF = col1.loc[index,"TCF"]
    ECF = col1.loc[index,"ECF"]
    Real_E = col1.loc[index,"Real_Effort"]
    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    #col1.at(5,"UUCW Algen", desired_UUCW , True)
    col1.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)


col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,576
2,8,4,20,15,445,0.99,0.99,398.1,7962,398
3,9,1,5,20,355,1.03,0.8,396.75,7935,472
4,8,5,10,15,350,0.9,0.91,390.25,7805,468
5,8,1,10,16,345,0.9,0.91,387.9,7758,465
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,272
68,17,1,23,22,565,1.03,0.8,289.55,5791,334
69,19,5,30,10,475,0.95,0.92,289.1,5782,311
70,6,5,15,5,250,1,0.92,288.9,5778,308


In [6]:
array_aePerulangan = []
for index, row in col1.iterrows():
    print()
    print("____ DATA ",index,"____")

    #Setting Parameter
    c1 = 2
    c2 = 2
    r1 = (0, 1)
    r2 = (0, 1)
    iteration_max = 10 #40
    particles = 70
    fitness_value = 200
    productivity = 20
    inertia_max = 0.9
    inertia_min = 0.4
    inertia = 0
    temp = 0
    ae = 0
    particles_position = [0]
    simpleUC = col1.loc[index,"S"]
    averageUC = col1.loc[index,"A"]
    complexUC = col1.loc[index,"C"]
    estimated_effort = 0
    uaw = col1.loc[index,"UAW"]
    uucw = col1.loc[index,"UUCW"]
    tcf = col1.loc[index,"TCF"]
    ecf = col1.loc[index,"ECF"]
    real_effort = col1.loc[index,"Real_Effort"]
    array_aeParticle = []
    GBest = 5000
    particle = []

    #perhitungan UCP
    def ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf):
        ucSimple = xSimple * simpleUC
        ucAverage = xAverage * averageUC
        ucComplex = xComplex * complexUC
    
        uucw = ucSimple + ucAverage + ucComplex
        uucp = uaw + uucw
        return uucp * tcf * ecf

    #Generate random Simple Use Case Complexity Weight Parameter
    def randomSimpleUCW(lower_bound=5, upper_bound=7.49, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Average Use Case Complexity Weight Parameter 
    def randomAverageUCW(lower_bound=7.5, upper_bound=12.49, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Complex Use Case Complexity Weight Parameter
    def randomComplexUCW(lower_bound=12.5, upper_bound=15, n_particles=10):
        #inisialisasi posisi awal
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]
    
    for i in range(iteration_max):
        
        #Hitung inertia
        inertia = inertia_max - ((inertia_max-inertia_min)*i/iteration_max)
        print('Perulangan ',i)
        print('inertia :', inertia)
        
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)
        print('r1 :', r1)
        print('r2 :', r2)
        print()
        
        if i == 0:
            for j in range(particles):    
                xSimple = randomSimpleUCW()
                xAverage = randomAverageUCW()
                xComplex = randomComplexUCW()
                #case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
                #estimated_effort = case * 20
                        
                #print('Partikel ke', j,':',estimated_effort)
                #print('ucp   ', case)
                #print('ae ', j, ae)
                #print('S     ', xSimple)
                #print('A     ', xAverage)
                #print('C     ', xComplex)
                #print()
            
                #print('estimated effort', estimated_effort)
            
                if j == 0:
                    pbest_simple = xSimple
                    pbest_average = xAverage
                    pbest_complex = xComplex
                
                    vSimple = random.uniform(0, 1)
                    vAverage = random.uniform(0, 1)
                    vComplex = random.uniform(0, 1)
                
                    xSimple = pbest_simple
                    xAverage = pbest_average
                    xComplex = pbest_complex
                    
                    case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
                    estimated_effort = case * 20
                    
                    #Selisih estimasi effort
                    ae = abs(estimated_effort - real_effort)
                    
                    #Menyimpan GBest
                    if GBest > ae:
                        GBest = ae
                        letak_i = i
                        letak_j = j
                        xSimpleMin = xSimple
                        xAverageMin = xAverage
                        xComplexMin = xComplex
                    
                    #Simpan nilai ke dalam Array
                    particle = [[xSimple, xAverage, xComplex, vSimple, vAverage, vComplex, ae]]
                    print('particle', j,':', particle)
                    print()
                    
                PBest = particle[0][0]
                
                if j > 0:
                    #Inisialisasi Personal Best awal
                    pbest_simple = xSimple
                    pbest_average = xAverage
                    pbest_complex = xComplex
                    
                    #Inisialisasi velocity awal
                    vSimple = random.uniform(0, 1)
                    vAverage = random.uniform(0, 1)
                    vComplex = random.uniform(0, 1)
                
                    xSimple = pbest_simple
                    xAverage = pbest_average
                    xComplex = pbest_complex
                    
                    #Menghitung  estimasi effort
                    case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
                    estimated_effort = case * 20
                    
                    #Selisih estimasi effort
                    ae = abs(estimated_effort - real_effort)
                    print(ae)
                    
                    #Menyimpan GBest
                    if GBest > ae:
                        GBest = ae
                        letak_i = i
                        letak_j = j
                        xSimpleMin = xSimple
                        xAverageMin = xAverage
                        xComplexMin = xComplex
                    
                    #Simpan nilai ke dalam Array
                    particle = [[xSimple, xAverage, xComplex, vSimple, vAverage, vComplex, ae]]
                    print('particle', j,':', particle)

                    #Hitung velocity
                    vSimple = inertia * vSimple + (c1 * r1) * (pbest_simple - xSimple) + (c2* r2) * (GBest - xSimple)
                    print("vSimple :", vSimple)
                    vAverage = inertia * vAverage + (c1 * r1) * (pbest_average - xAverage) + (c2* r2) * (GBest - xAverage)
                    print("vAverage :", vAverage)
                    vComplex = inertia * vComplex + (c1 * r1) * (pbest_complex - xComplex) + (c2* r2) * (GBest - xComplex)
                    print("vComplex :", vComplex)
                    print()
                    
                if GBest < 100:
                    break
            
            #Memasukkan selisih
            #array_aeParticle.append(ae)
            #aeParticle_min = np.min(array_aeParticle)
            #index_of_minimum = np.where(array_aeParticle == aeParticle_min)
        
            #menjumlah 10 angk a selisih
            #temp = np.array(temp+ae)
         
            #print('Partikel ke', j,':',estimated_effort)
            #print('ucp   ', case)
            #print('ae ', j, ae)
            #print('S     ', xSimple)
            #print('A     ', xAverage)
            #print('C     ', xComplex)
            #print()
            
            #print('estimated effort', estimated_effort)
            
        #print('ae terkecil : ',np.min(aeParticle_min))
        #print('letak ae : ',index_of_minimum)
        #print('pada perulangan ke :', aeParticle_min.index(min(aeParticle_min)))
        #mae = temp/particle
        #print('MAE :', mae)
        print()
        #array_aePerulangan.append(aeParticle_min)
    print('GBest :', GBest)
    print('iterasi ', letak_i)
    print('particle', letak_j)
    print('S Min', xSimpleMin)
    print('A Min', xAverageMin)
    print('C Min', xComplexMin)  


____ DATA  1 ____
Perulangan  0
inertia : 0.9
r1 : 0.4709806607428503
r2 : 0.11450395548334757

particle 0 : [[7.012284480376086, 8.500648576890622, 12.620843979558972, 0.7556431412607346, 0.1744207897352259, 0.9548032186372083, 3542.0540652953987]]

2734.176017357655
particle 1 : [[6.086778764312183, 11.464330161559035, 14.973116900395997, 0.5911587967326527, 0.50747915700827, 0.4631388036650369, 2734.176017357655]]
vSimple : 625.2860603780427
vAverage : 623.9792468906912
vComplex : 623.1358006515937

3288.6406047889604
particle 2 : [[5.028957814717854, 11.707314852597941, 12.517889684565104, 0.4819816002686683, 0.5817796372435028, 0.8545180539852979, 3288.6406047889604]]
vSimple : 625.4300502670685
vAverage : 623.9904719064112
vComplex : 624.050308432528

3335.0259391214986
particle 3 : [[5.434795889911088, 8.737134245691271, 14.108429443001038, 0.5528563460394055, 0.8649128384537805, 0.8637994023779101, 3335.0259391214986]]
vSimple : 625.4008974084716
vAverage : 624.925486643482
vC


Perulangan  4
inertia : 0.7
r1 : 0.18487395932275297
r2 : 0.462313380802974


Perulangan  5
inertia : 0.65
r1 : 0.7588074992862381
r2 : 0.6913514520440196


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.5452009850297501
r2 : 0.8301327515742071


Perulangan  7
inertia : 0.55
r1 : 0.008269206954509722
r2 : 0.7356607728118516


Perulangan  8
inertia : 0.5
r1 : 0.13976768443637333
r2 : 0.8296153797573003


Perulangan  9
inertia : 0.45
r1 : 0.7543415840805474
r2 : 0.9657969115482596


GBest : 1790.1397762731103
iterasi  0
particle 44
S Min 7.288898203624783
A Min 11.756925566647023
C Min 14.889710299270138

____ DATA  4 ____
Perulangan  0
inertia : 0.9
r1 : 0.5497258586255633
r2 : 0.6972597954505516

particle 0 : [[7.235858235589862, 10.952562151123484, 13.27181264951772, 0.8758724276477281, 0.656343648294048, 0.8374919044108017, 2026.429162164659]]

2045.9371522265446
particle 1 : [[5.4954971907757875, 9.666156405733606, 14.630139229099527, 0.35906286495557127, 0.28214380766158653, 0.

1950.2364177180234
particle 31 : [[6.598124723825135, 10.609069997864243, 14.617218535010519, 0.9245465841377128, 0.48058053382645016, 0.21271723279946175, 1950.2364177180234]]
vSimple : 1601.8718478036299
vAverage : 1597.5465757049064
vComplex : 1593.3825333797768

2398.9316946337376
particle 32 : [[5.269617413881837, 10.959553151880588, 12.769143642585615, 0.5304731764335204, 0.18331166281815447, 0.7394051136987079, 2398.9316946337376]]
vSimple : 1602.8174549429946
vAverage : 1596.9359992107518
vComplex : 1595.665351150522

2595.2066825935235
particle 33 : [[5.983081993232304, 8.458255681656524, 13.538950525419693, 0.8865374731410949, 0.7775977232503154, 0.8364302961054362, 2595.2066825935235]]
vSimple : 1602.4396111377494
vAverage : 1599.9189953070374
vComplex : 1594.9992272538132

1875.4519937501955
particle 34 : [[6.037743761166254, 12.204258479284972, 13.940599147340595, 0.20298261314215482, 0.8543871302310472, 0.4014789375863409, 1875.4519937501955]]
vSimple : 1601.7709116951232

vAverage : 47.48919154029567
vComplex : 47.104144254551244

1378.3922874242007
particle 14 : [[6.443756099887562, 12.150351313848184, 13.839821390272599, 0.6276937604521661, 0.7536087121967607, 0.3233564409592967, 1378.3922874242007]]
vSimple : 47.570504823797826
vAverage : 47.47066648023605
vComplex : 47.02033166883604

2542.8786855155586
particle 15 : [[7.391674252064751, 7.94922279157627, 13.552944219263805, 0.6969528209515355, 0.27060720949739225, 0.6514866265527156, 2542.8786855155586]]
vSimple : 47.597429835191754
vAverage : 47.19289234707078
vComplex : 47.326364727202204

2144.135010216718
particle 16 : [[7.1363545749675055, 8.949696971812287, 14.235279592717525, 0.9431365184532209, 0.07029492736887855, 0.7986208074904259, 2144.135010216718]]
vSimple : 47.82853226868471
vAverage : 46.975239993939276
vComplex : 47.433297816390365

2163.774559661035
particle 17 : [[6.473877401669121, 8.913630104280056, 14.184058729987864, 0.48563028579954204, 0.7764695620927833, 0.4600362553215952

vSimple : 1166.2711148040726
vAverage : 1158.9726190823994
vComplex : 1150.8907774063155

1332.450604396703
particle 60 : [[6.718278989852436, 8.438038749458796, 14.210136452469259, 0.11523478042765245, 0.8211336829683975, 0.012097177563255346, 1332.450604396703]]
vSimple : 1166.0957006520869
vAverage : 1163.2960829909837
vComplex : 1151.0391698801948

1324.2141431225828
particle 61 : [[7.054289977324538, 8.84826454644898, 13.920485739952758, 0.7954003118216144, 0.6203979249261706, 0.582207124521622, 1324.2141431225828]]
vSimple : 1166.0367250171664
vAverage : 1162.296064776996
vComplex : 1152.1307967069652

806.049499559429
particle 62 : [[5.0181394601951, 12.002115873500884, 13.89083950758114, 0.035806539973699536, 0.5634366302085935, 0.31235395078387185, 806.049499559429]]
vSimple : 1169.4199538475884
vAverage : 1155.9455197346138
vComplex : 1151.9471421411608

1566.840975941349
particle 63 : [[5.2814927594819085, 8.648710820577795, 13.446385853125445, 0.8485357193550576, 0.09486009

...

vSimple : 25.950125983614093
vAverage : 26.039701804737334
vComplex : 25.733576678428697

3593.915934935987
particle 48 : [[5.299851356679662, 8.237549392908322, 14.035419503672301, 0.2025521962714245, 0.7837894198096284, 0.6742978525767126, 3593.915934935987]]
vSimple : 25.517723601112174
vAverage : 26.013883959842982
vComplex : 25.86214655886955

4844.003221390471
particle 49 : [[5.089195865261534, 11.307559780999313, 14.461804326646073, 0.47862492421438885, 0.5105750052157088, 0.8119494797830428, 4844.003221390471]]
vSimple : 25.76812180339782
vAverage : 25.739823889102198
vComplex : 25.982120977007423

3608.9738829727557
particle 50 : [[6.567702786665112, 8.421383678626668, 13.725170937678397, 0.3369430074042914, 0.7082067459949015, 0.5703814653034184, 3608.9738829727557]]
vSimple : 25.627042896558514
vAverage : 25.944172888670977
vComplex : 25.77146831588411

3097.5524114314594
particle 51 : [[7.077177135221417, 7.904147068680974, 12.859317819154494, 0.30934988993401746, 0.9710859

Perulangan  0
inertia : 0.9
r1 : 0.6761902919147317
r2 : 0.30636789443581913

particle 0 : [[5.531125490721555, 12.138254442812315, 14.665234768505439, 0.41829887058316806, 0.799663242466891, 0.2998266112757648, 4387.774174410528]]

3646.144411661602
particle 1 : [[5.851657138825843, 10.904487029042636, 13.838117844371357, 0.6681642492944512, 0.8199545315789042, 0.9627250200979922, 3646.144411661602]]
vSimple : 2231.139000490587
vAverage : 2228.1795620358275
vComplex : 2226.510514883606

3540.5984813274335
particle 2 : [[6.385717118680504, 10.606156851291503, 13.78583614235758, 0.4998966380259737, 0.14016515089293513, 0.21040933066761514, 3540.5984813274335]]
vSimple : 2165.988553091887
vAverage : 2163.0787802845207
vComplex : 2161.1936967475476

4127.173012642503
particle 3 : [[6.453791296724709, 11.785088131934646, 14.132370160614785, 0.27092712873505287, 0.5825650577935455, 0.44832828528882285, 4127.173012642503]]
vSimple : 2165.740769048339
vAverage : 2162.754566812461
vComplex : 2

vSimple : 2817.902099033013
vAverage : 2811.656061297803
vComplex : 2807.0815417519807

2919.642495505277
particle 22 : [[5.932933647576989, 8.66318921446398, 13.926005376055773, 0.36004335707839896, 0.882776984318872, 0.7329198823073519, 2919.642495505277]]
vSimple : 2817.320324583151
vAverage : 2814.674368723752
vComplex : 2808.532321541477

3157.234683090719
particle 23 : [[5.288281231059933, 8.794868575532327, 14.47296122995664, 0.5584708549487796, 0.5923602393029326, 0.11059673855254115, 3157.234683090719]]
vSimple : 2818.2347397822446
vAverage : 2814.2626898908366
vComplex : 2807.347914782542

4018.361277308177
particle 24 : [[5.164200641261888, 11.118142413956459, 14.424852576550425, 0.7069138957419808, 0.610082111154273, 0.20725809358885405, 4018.361277308177]]
vSimple : 2818.5099687587804
vAverage : 2811.6267676635302
vComplex : 2807.489823023768

2673.914031194203
particle 25 : [[5.6220888185187805, 8.30451303635291, 13.63735488240789, 0.8265796633465724, 0.07851534155445183,

vSimple : 1179.216037440892
vAverage : 1169.7933840096227
vComplex : 1165.0599848008665

1254.3128522036113
particle 2 : [[7.1744686373753845, 9.06724997122251, 13.594293925004779, 0.06902596844651221, 0.8998530222508342, 0.7426517369144664, 1254.3128522036113]]
vSimple : 1176.0133783256972
vAverage : 1173.7595624648743
vComplex : 1166.4391247354358

1054.3191079601847
particle 3 : [[5.111456713035991, 10.133389981591606, 14.632730864361095, 0.09176059575521744, 0.43079026289169964, 0.8588422830646536, 1054.3191079601847]]
vSimple : 1179.3053502520136
vAverage : 1171.6467281543905
vComplex : 1164.8969497479952

1156.9638993152448
particle 4 : [[7.467145261990843, 9.15619690790424, 14.092917282608994, 0.5860553176591146, 0.20659950161109675, 0.6374809458189128, 1156.9638993152448]]
vSimple : 1176.0145800706919
vAverage : 1172.9945828282266
vComplex : 1165.5537573774398

1658.3716344111754
particle 5 : [[5.65819394891841, 8.111930985407586, 13.58458228604171, 0.8848376193359544, 0.683092

particle 53 : [[6.529188548392945, 9.856743319727995, 14.97793616999121, 0.09214663904454157, 0.5257896553861168, 0.8947989632551497, 1528.1335516640875]]
vSimple : 170.3777713084687
vAverage : 168.83381013547523
vComplex : 166.18907330209862

1357.52312107727
particle 54 : [[6.591272436253603, 11.418558916488234, 12.642677290767386, 0.042404295274451775, 0.07591276608484865, 0.38071413219903594, 1357.52312107727]]
vSimple : 170.29691509819204
vAverage : 167.5210693046785
vComplex : 167.0838354285538

722.3282416587135
particle 55 : [[5.565450127959705, 9.130972451880327, 12.89698262872156, 0.7424523213090096, 0.09998655934625156, 0.5938413758599236, 722.3282416587135]]
vSimple : 171.52324795853252
vAverage : 168.8724631883239
vComplex : 167.12782742887356

341.0691640943869
particle 56 : [[5.05442770372032, 7.7603799417726895, 13.01621179178714, 0.33366285487305747, 0.9402497321508431, 0.33008665081340993, 341.0691640943869]]
vSimple : 171.4523843769268
vAverage : 170.42539758453816
v